<a href="https://colab.research.google.com/github/Jefferson-G/Proyecto-SQL-avanzado-ETL/blob/main/%5BNRC_8903%5D_JeffersonGuam%C3%A1n_ProyectoFinalUnidad2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#librerias
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy import inspect

In [2]:
#Conectar el motor al archivo de la base de datos a usar
engine = create_engine('sqlite:///chinook.db')
engine

Engine(sqlite:///chinook.db)

# Extract

In [3]:
# SQL Expression Language crea metadatos que contienen objetos que definen la tabla de empleados
metadata = MetaData()

# Este método instancia las tablas que ya
# existe en la base de datos, a la que está conectado el motor.
metadata.create_all(engine)

# Revisando esto, podemos ver la estructura de la tabla y los tipos de variables para la tabla de employees
inspector = inspect(engine)

#Revisé las columnas en la tabla de employees 
inspector.get_columns('employees')

[{'autoincrement': 'auto',
  'default': None,
  'name': 'EmployeeId',
  'nullable': False,
  'primary_key': 1,
  'type': INTEGER()},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'LastName',
  'nullable': False,
  'primary_key': 0,
  'type': NVARCHAR(length=20)},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'FirstName',
  'nullable': False,
  'primary_key': 0,
  'type': NVARCHAR(length=20)},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'Title',
  'nullable': True,
  'primary_key': 0,
  'type': NVARCHAR(length=30)},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'ReportsTo',
  'nullable': True,
  'primary_key': 0,
  'type': INTEGER()},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'BirthDate',
  'nullable': True,
  'primary_key': 0,
  'type': DATETIME()},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'HireDate',
  'nullable': True,
  'primary_key': 0,
  'type': DATETIME()},
 {'autoincrement': 'auto',
  'default': None,
  'name

# Transform

In [4]:
#seleccionar todos los campos de la tabla Employees

# Ejecutemos SQL sin procesar en algunas tablas usando SQLAlchemy
with engine.connect() as con:
    
    rs = con.execute('SELECT * FROM employees')
    
    for row in rs:
        print(row)
# No olvide cerrar su conexión a la base de datos cuando finalice la consulta
con.close()

# Nota: Use LIMIT n para limitar el número de filas devueltas. No hay problema con esta pequeña mesa.

(1, 'Adams', 'Andrew', 'General Manager', None, '1962-02-18 00:00:00', '2002-08-14 00:00:00', '11120 Jasper Ave NW', 'Edmonton', 'AB', 'Canada', 'T5K 2N1', '+1 (780) 428-9482', '+1 (780) 428-3457', 'andrew@chinookcorp.com')
(2, 'Edwards', 'Nancy', 'Sales Manager', 1, '1958-12-08 00:00:00', '2002-05-01 00:00:00', '825 8 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 2T3', '+1 (403) 262-3443', '+1 (403) 262-3322', 'nancy@chinookcorp.com')
(3, 'Peacock', 'Jane', 'Sales Support Agent', 2, '1973-08-29 00:00:00', '2002-04-01 00:00:00', '1111 6 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 5M5', '+1 (403) 262-3443', '+1 (403) 262-6712', 'jane@chinookcorp.com')
(4, 'Park', 'Margaret', 'Sales Support Agent', 2, '1947-09-19 00:00:00', '2003-05-03 00:00:00', '683 10 Street SW', 'Calgary', 'AB', 'Canada', 'T2P 5G3', '+1 (403) 263-4423', '+1 (403) 263-4289', 'margaret@chinookcorp.com')
(5, 'Johnson', 'Steve', 'Sales Support Agent', 2, '1965-03-03 00:00:00', '2003-10-17 00:00:00', '7727B 41 Ave', 'Calgary', 'A

In [5]:
#-- Consultar  el total de factura que an generado los agentes de Ventas desde una fecha inico y una fecha fin,
#junto con que pais de los clientes se asociaron mas,  y que grupo de artistas resulto mas vendido junto con su album y
#tipo de genero musical. 
with engine.connect() as con:
    
    rs = con.execute("""select e.FirstName||" "|| e.LastName as Agente_Ventas,
c.Country Ciudad , at.name as Grupo, a.Title as Album, 
g.Name as Tipo_Género,
count(*) as TotalFacturas, min(InvoiceDate) as FechaMin,
max(InvoiceDate) as FechaMax, sum(Total) as ValorTotal
from invoices i
inner join customers c on c.CustomerId = i.CustomerId
inner join employees e on c.SupportRepId = e.EmployeeId
inner join invoice_items it on i.InvoiceId= it.InvoiceId
inner join tracks t on it.TrackId = t.TrackId
inner join genres g on t.GenreId = g.GenreId
inner join albums a on t.AlbumId = a.AlbumId
inner join artists at on a.ArtistId = at.ArtistId
group by Agente_Ventas
having ValorTotal > 3000
order by Agente_Ventas DESC;
""")
    
    for row in rs:
        print(row)
        
con.close()

('Steve Johnson', 'USA', 'U2', 'B-Sides 1980-1990', 'Rock', 684, '2009-01-01 00:00:00', '2013-12-05 00:00:00', 6490.159999999937)
('Margaret Park', 'Portugal', 'U2', 'Rattle And Hum', 'Rock', 760, '2009-01-02 00:00:00', '2013-12-09 00:00:00', 6931.39999999992)
('Jane Peacock', 'India', 'The Office', 'The Office, Season 1', 'TV Shows', 796, '2009-01-19 00:00:00', '2013-12-22 00:00:00', 7427.059999999916)


In [7]:
# Los 10 mejores clientes indicando Nombre, Apellidos, 
#Pais la cantidad de gasto que han realizado.
with engine.connect() as con:
    
    rs = con.execute("""select customers.FirstName as Nombre_Cliente,
customers.LastName as Apellido_Cliente, customers.Country as País_Cliente,
min(invoices.InvoiceDate) as Primera_Compra, max(invoices.InvoiceDate) as Ultima_Compra
, sum(invoices.Total) as Total_Gasto
from customers, invoices
where customers.CustomerId = invoices.CustomerId
group by customers.CustomerId
order by Total desc limit 10""")
    
    for row in rs:
        print(row)
        
con.close()

('Helena', 'Holý', 'Czech Republic', '2009-07-11 00:00:00', '2013-11-13 00:00:00', 49.620000000000005)
('Eduardo', 'Martins', 'Brazil', '2009-04-09 00:00:00', '2013-08-12 00:00:00', 37.620000000000005)
('Mark', 'Philips', 'Canada', '2009-01-06 00:00:00', '2013-05-11 00:00:00', 37.620000000000005)
('Patrick', 'Gray', 'USA', '2009-06-10 00:00:00', '2013-10-13 00:00:00', 37.620000000000005)
('Martha', 'Silk', 'Canada', '2009-03-09 00:00:00', '2013-07-12 00:00:00', 37.620000000000005)
('Terhi', 'Hämäläinen', 'Finland', '2009-08-11 00:00:00', '2013-12-14 00:00:00', 41.620000000000005)
('Johannes', 'Van der Berg', 'Netherlands', '2009-05-10 00:00:00', '2013-09-12 00:00:00', 40.62)
('Emma', 'Jones', 'United Kingdom', '2009-02-06 00:00:00', '2013-06-11 00:00:00', 37.620000000000005)
('Jack', 'Smith', 'USA', '2009-03-04 00:00:00', '2012-07-31 00:00:00', 39.620000000000005)
('João', 'Fernandes', 'Portugal', '2009-05-05 00:00:00', '2012-10-01 00:00:00', 39.620000000000005)


# Load

In [8]:
#Cargar a última query. Aquí debe utilizar un objeto dataframe
import pandas as pd
df = pd.read_sql_query("""select e.FirstName||" "|| e.LastName as Agente_Ventas,
c.Country Ciudad , at.name as Grupo, a.Title as Album, 
g.Name as Tipo_Género,
count(*) as TotalFacturas, min(InvoiceDate) as FechaMin,
max(InvoiceDate) as FechaMax, sum(Total) as ValorTotal
from invoices i
inner join customers c on c.CustomerId = i.CustomerId
inner join employees e on c.SupportRepId = e.EmployeeId
inner join invoice_items it on i.InvoiceId= it.InvoiceId
inner join tracks t on it.TrackId = t.TrackId
inner join genres g on t.GenreId = g.GenreId
inner join albums a on t.AlbumId = a.AlbumId
inner join artists at on a.ArtistId = at.ArtistId
group by Agente_Ventas
having ValorTotal > 3000
order by Agente_Ventas DESC;
    
    """, con=engine.connect())

In [9]:
df.head()

,Agente_Ventas,Ciudad,Grupo,Album,Tipo_Género,TotalFacturas,FechaMin,FechaMax,ValorTotal
0,Steve Johnson,USA,U2,B-Sides 1980-1990,Rock,684,2009-01-01 00:00:00,2013-12-05 00:00:00,6490.16
1,Margaret Park,Portugal,U2,Rattle And Hum,Rock,760,2009-01-02 00:00:00,2013-12-09 00:00:00,6931.40
2,Jane Peacock,India,The Office,"The Office, Season 1",TV Shows,796,2009-01-19 00:00:00,2013-12-22 00:00:00,7427.06


In [11]:
#Exportar el archivo en un formato a su elección.
df.to_csv('[NRC_8903] JeffersonGuamán_ProyectoFinalUnidad1.csv')